<strong>1. Load mô hình và các thư viện cần dùng</strong>

In [42]:
from collections import Counter
from keras.models import load_model # type: ignore
import nltk
import numpy as np
import re

model = load_model("model_0.9972.h5")

NGRAM = 5
MAXLEN = 39

# Các ký tự có thể xuất hiện trong encoder
alphabet = ['\x00', ' '] + list('0123456789aàáảãạăằắẳẵặâầấẩẫậeèéẻẽẹêềếểễệiìíỉĩịoòóỏõọôồốổỗộơờớởỡợuùúủũụưừứửữựyỳýỷỹỵAÀÁẢÃẠĂẰẮẲẴẶÂẦẤẨẪẬEÈÉẺẼẸÊỀẾỂỄỆIÌÍỈĨỊOÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢUÙÚỦŨỤƯỪỨỬỮỰYỲÝỶỸỴbcdfghjklmnpqrstvwxzBCDFGHJKLMNPQRSTVWXZđĐ')

<strong>2. Hàm encoder và decoder tương ứng với mô hình đã huấn luyện</strong>

In [43]:
# Đệm '\x00' vào cuối của các cụm ngrams có độ dài < độ dài tối đa là 39
def _encoder_data(text):
    x = np.zeros((MAXLEN, len(alphabet)))
    for i, c in enumerate(text[:MAXLEN]):
        x[i, alphabet.index(c)] = 1
    if i < MAXLEN - 1 :
        for j in range(i + 1, MAXLEN):
            x[j, 0] = 1
    return x
def _decoder_data(x):
    x = x.argmax(axis = -1)
    return ''.join(alphabet[i] for i in x)

In [44]:
print(_encoder_data('Xử lý ngôn ngữ').shape)
print(_decoder_data(_encoder_data('Xử lý ngôn ngữ')))

(39, 198)
Xử lý ngôn ngữ                         


<strong>3. Các hàm sẽ sử dụng để sửa lỗi chính tả trong văn bản Tiếng Việt</strong>

- Hàm tác câu thành các ngrams

In [45]:
def _nltk_ngrams(sentence, n, maxlen):
    list_ngrams = []
    list_words = sentence.split()
    num_words = len(list_words)

    if (num_words >= n):
        for ngram in nltk.ngrams(list_words, n):
            if len(' '.join(ngram)) <= maxlen:
                list_ngrams.append(ngram)
    else:
        list_ngrams.append(tuple(list_words))

    return list_ngrams

In [46]:
_nltk_ngrams('Xuwr ný ngoon ngữ', NGRAM, MAXLEN)

[('Xuwr', 'ný', 'ngoon', 'ngữ')]

- Hàm dự đoán ngram bằng mô hình

In [47]:
def _guess(ngram):
    text = ' '.join(ngram)
    preds = model.predict(np.array([_encoder_data(text)]))
    return _decoder_data(preds[0]).strip('\x00')

In [48]:
_guess(('Xuwr', 'ný', 'ngoon', 'ngữ'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


'Xử lý ngôn ngữ'

- Hàm thêm dấu câu khi sửa lỗi chính tả

In [49]:
def _add_punctuation(text, corrected_text):

    # Lưu dấu câu theo chỉ số của từ
    list_punctuation = {}

    # Thu thập dấu câu cho mỗi từ trong `text`
    for (i, word) in enumerate(text.split()):
        if word[0] not in alphabet or word[-1] not in alphabet:
            # Kiểm tra dấu câu ở đầu từ
            start_punc = ''
            for c in word:
                if c in alphabet:
                    break
                start_punc += c

            # Kiểm tra dấu câu ở cuối từ
            end_punc = ''
            for c in word[::-1]:
                if c in alphabet:
                    break
                end_punc += c
            end_punc = end_punc[::-1]

            # Lưu dấu câu vào từ điển
            list_punctuation[i] = [start_punc, end_punc]

    # Thêm dấu câu vào `corrected_text`
    result = ''
    for (i, word) in enumerate(corrected_text.split()):
        if i in list_punctuation:
            result += (list_punctuation[i][0] + word + list_punctuation[i][1]) + ' '
        else:
            result += word + ' '
            
    return result.strip()


- Hàm sửa lỗi chính tả trong câu

In [50]:
import re
from collections import Counter

def _correct(text):
    # Xóa ký tự đặc biệt
    new_text = re.sub(r'[^' + ''.join(alphabet) + ']', '', text)
    
    ngrams = list(_nltk_ngrams(new_text, NGRAM, MAXLEN))
    guessed_ngrams = [_guess(ngram) for ngram in ngrams]
    candidates = [Counter() for _ in range(len(guessed_ngrams) + NGRAM - 1)]
    
    for nid, ngram in (enumerate(guessed_ngrams)):
        for wid, word in (enumerate(re.split(r'\s', ngram))):
            candidates[nid + wid].update([word])
    
    corrected_text = ' '.join(c.most_common(1)[0][0] for c in candidates if c)
    return _add_punctuation(text, corrected_text)


In [51]:
text = 'A đax nói rằng: "Chong thowfi đại soos hosa hiện lay, văn bản đánh máy ddax daafn thay thế vawn barn viết tay bởi sự thuận tieejn của nó."'
_correct(text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


'Anh đã nói rằng: "Trong thời đại số hóa hiện nay, văn bản đánh máy đã dần thay thế văn bản viết tay bởi sự thuận tiện của nó."'

Thủ tướng Phạm Minh Chính sẽ tham dự Hội nghị Thượng đỉnh Hợp tác Tiểu vùng Mekong mở rộng và một số hội nghị cấp cao tại Trung Quốc.
Chuyến công tác của Thủ tướng Phạm Minh Chính sẽ diễn ra ngày 5-8/11 theo lời mời của Thủ tướng Trung Quốc Lý Cường, theo thông cáo của Bộ Ngoại giao.
Ngoài Hội nghị Thượng đỉnh Hợp tác Tiểu vùng Mekong mở rộng (GMS) lần thứ 8, Thủ tướng Phạm Minh Chính cũng sẽ tham dự Hội nghị Cấp cao Chiến lược hợp tác kinh tế Ayeyawady - Chao Phraya - Mekong lần thứ 10, Hội nghị Cấp cao Hợp tác Campuchia - Lào - Myanmar - Việt Nam lần thứ 11 và làm việc tại Trung Quốc.
GMS là khu vực được gắn kết với nhau bởi sông Mekong, có diện tích 2,6 triệu km vuông. GMS được đánh giá là cơ chế hợp tác có hiệu quả hàng đầu trong các cơ chế hợp tác tại tiểu vùng. Các bên tham gia cơ chế này gồm Việt Nam, Lào, Campuchia, Myanmar, Thái Lan và hai tỉnh Vân Nam, Quảng Tây của Trung Quốc.
Phát biểu tại Hội nghị Thượng đỉnh Hợp tác Tiểu vùng Mekong mở rộng lần thứ 7 hồi tháng 9/2021, Thủ tướng Phạm Minh Chính nêu các nội dung chính trong hợp tác của GMS, trong đó có bảo đảm chuỗi cung ứng khu vực không bị đứt gãy; tạo đột phá về xây dựng hệ thống kết cấu hạ tầng đồng bộ; xây dựng GMS xanh, an toàn và thích ứng hiệu quả với biến đổi khí hậu; tăng cường phối hợp giữa GMS với ASEAN và các cơ chế khu vực khác.
Trung Quốc liên tục là đối tác thương mại lớn nhất của Việt Nam và thị trường xuất khẩu lớn thứ hai của Việt Nam. Việt Nam là đối tác thương mại lớn nhất của Trung Quốc trong ASEAN và đối tác thương mại lớn thứ 5 của Trung Quốc trên thế giới.
Năm 2023, tổng kim ngạch thương mại hai nước đạt 171,9 tỷ USD. Thương mại 8 tháng đầu năm đạt 130,78 tỷ USD, tăng 25%.

<strong>4. Kết quả khi áp dụng mô hình</strong>

In [81]:
# Nhập văn bản
text = input("Nhập văn bản: ")

# Tách văn bản thành từng từ và dấu câu
words_with_punct = re.findall(r'\b\w+\b|[^\w\s]', text)

# Tạo danh sách các từ đã được sửa
corrected_words_with_punct = [(_correct(word) if re.match(r'\w+', word) else word) for word in words_with_punct]

# Kết hợp các từ lại thành chuỗi kết quả với dấu câu giữ nguyên
result = ''.join([word if re.match(r'[^\w\s]', word) else ' ' + word for word in corrected_words_with_punct]).strip()

# In kết quả văn bản sau khi sửa
print("Văn bản sau khi sửa lỗi chính tả:", result)

# Lọc ra các từ đã được sửa
corrected_word = [(words_with_punct[i], corrected_words_with_punct[i]) 
                  for i in range(len(words_with_punct)) 
                  if words_with_punct[i] != corrected_words_with_punct[i] and re.match(r'\w+', words_with_punct[i])]

# Hiển thị danh sách từ đã sửa
print("Các từ đã sửa:", corrected_word)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━

Thur tướng Phạm Minh Chính sẽ tham dự Hội nghị Thượng đỉnh Hợp tác Tiểu vùng Mekong mowr rộng và một số hooji nghị cấp cao taij Trung Quốc. Chuyến coong tác của Thủ tướng Phạm Minh Chính sẽ diễn ra ngafy 5-8/11 theo lời mowfi của Thủ tướng Trung Quốc Lý Cường, theo thoong cáo của Bộ Ngoại giao. Ngoafi Hội nghị Thượng đỉnh Hợp tác Tiểu vùng Mekong mở rộng (GMS) laafn thuws 8, Thủ tướng Phạm Minh Chính cuxng sẽ tham dự Hội nghị Cấp cao Chiến lược hợp tasc kinh tế Ayeyawady - Chao Phraya - Mekong lần thứ 10, Hội nghị Cấp cao Hợp tác Campuchia - Lào - Myanmar - Việt Nam lần thứ 11 và làm việc tại Trung Quốc. GMS laf khu vực được gắn kết với nhau bởi soong Mekong, cos diện tích 2,6 triệu km vuông. GMS được đánh giá là cơ chees hợp tác có hieeju quả hàng đầu trong các cơ chế hợp tác tại tiểu vùng. Các bên tham gia cơ chế này gồm Việt Nam, Lào, Campuchia, Myanmar, Thái Lan và hai tỉnh Vân Nam, Quảng Tây của Trung Quốc. Phast biểu tại Hội nghị Thượng đỉnh Hợp tác Tiểu vufng Mekong mở rộng lần thứ 7 hồi tháng 9/2021, Thủ tướng Phạm Minh Chính nêu các nội dung chisnh trong hợp tác của GMS, trong đó cos bảo đảm chuỗi cung ứng khu vực không bị đứt gaaxy; tạo đột phá veef xây dựng hệ thống keest cấu hạ tầng đồng booj; xaay dựng GMS xanh, an toafn và thisch ứng hiệu quar với biến đổi khí hậu; tawng cường phối hợp giữa GMS với ASEAN và các cơ chế khu vực khasc. Trung Quốc lieen tục là đối tác thương mại lớn nhất của Việt Nam và thị trường xuất khaaru lớn thứ hai của Việt Nam. Việt Nam là đối tác thương mại lớn nhất của Trung Quốc trong ASEAN và đối tác thương mại lớn thứ 5 của Trung Quốc trên thế giới. Năm 2023, tổng kim ngạch thương mại hai nước đạt 171,9 tỷ USD. Thương mại 8 thasng đầu nawm đạt 130,78 tyr USD, tăng 25%.

In [79]:
# Nhập văn bản
text = input("Nhập văn bản: ")

# Tách văn bản thành từng từ và dấu câu
words_with_punct = re.findall(r'\b\w+\b|[^\w\s]', text)

# Tạo danh sách các từ đã được sửa
corrected_words_with_punct = [(_correct(word) if re.match(r'\w+', word) else word) for word in words_with_punct]

# Kết hợp các từ lại thành chuỗi kết quả với dấu câu giữ nguyên
result = ''.join([word if re.match(r'[^\w\s]', word) else ' ' + word for word in corrected_words_with_punct]).strip()

# In kết quả văn bản sau khi sửa
print("Văn bản sau khi sửa lỗi chính tả:", result)

# Lọc ra các từ đã được sửa
corrected_word = [(words_with_punct[i], corrected_words_with_punct[i]) 
                  for i in range(len(words_with_punct)) 
                  if words_with_punct[i] != corrected_words_with_punct[i] and re.match(r'\w+', words_with_punct[i])]

# Hiển thị danh sách từ đã sửa
print("Các từ đã sửa:", corrected_word)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━

In [82]:
# Lưu văn bản ra file text
with open("corrected_text.txt", "w", encoding="utf-8") as file:
    file.write(result)